目的:
    這本筆記本的目的是處理空值,因為Price欄位和Mileage欄位都有空值

方法:
    使用的補空值方式是 RandomForestRegressor (from sklearn.ensemble import RandomForestRegressor)
    用 LabelEncoder (from sklearn import preprocessing) 將 類別型態的資料轉成純數字 , 但是這樣會有順序問題(數字有大小,但類別之間其實
    沒有大小關係)

結論:
    在 X 包含所有欄位(包括'Unnamed: 0'和'Title')的情況下 , 預測出來的 r平方最高 0.8125
    去掉 'Unnamed: 0'和'Title' , 皆會造成 r 平方下降
    標準化X的資料,可以讓 r平方 稍微上升, 從 0.7855 變成 0.7886

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('gurus_ML.csv')

In [3]:
data.columns

Index(['Unnamed: 0', 'Title', 'Year', 'Brand', 'Location', 'State', 'Rental',
       'Price', 'Mileage', 'Transmission', 'FuelType', 'Engine', 'Drivetrain',
       'OptionCount', 'MajorOptions', 'Accident Check', 'OwnershipHistory'],
      dtype='object')

------------------------

### 1.把資料切成三份

In [4]:
data.notnull().all(axis=1)

0           True
1           True
2           True
3           True
4           True
           ...  
2088473    False
2088474     True
2088475    False
2088476    False
2088477    False
Length: 2088478, dtype: bool

In [5]:
data1 = data[data.notnull().all(axis=1)]
# data1: 是每個欄位都有值的那些資料 , 之後會再被切割成兩份 ,一份訓練資料 ,一份測試資料

In [6]:
data1.isnull().sum()

Unnamed: 0          0
Title               0
Year                0
Brand               0
Location            0
State               0
Rental              0
Price               0
Mileage             0
Transmission        0
FuelType            0
Engine              0
Drivetrain          0
OptionCount         0
MajorOptions        0
Accident Check      0
OwnershipHistory    0
dtype: int64

In [7]:
data['Price'].isnull()

0          False
1          False
2          False
3          False
4          False
           ...  
2088473     True
2088474    False
2088475    False
2088476    False
2088477    False
Name: Price, Length: 2088478, dtype: bool

In [8]:
data2 = data[data['Price'].isnull()]
# data2 是價格為空值的那些資料 要被用來預測價格
# 裡面每一筆資料 , Price 都是 NaN

In [9]:
data['Mileage'].isnull()

0          False
1          False
2          False
3          False
4          False
           ...  
2088473    False
2088474    False
2088475     True
2088476     True
2088477     True
Name: Mileage, Length: 2088478, dtype: bool

In [10]:
data3 = data[data['Mileage'].isnull()]
# data3 是 Mileage 欄位是空值的那些資料
#每一筆資料的 Mileage都是 NaN

In [11]:
# 三份資料
# data1 : 全部資料都有值,之後還要切成訓練和測試資料
# data2 : Price 是空值的資料
# data3 : Mileage 是空值的資料

# 因為資料當中 , Price 和 Mileage 不會同時是空值, 所以 data2 和 data3 ,才不會出現兩個都是空值的狀況

---------------------

### 2.把類別資料轉成純數字

In [12]:
from sklearn import preprocessing

In [13]:
def transform_to_integer(i) :
    le = preprocessing.LabelEncoder()
    le.fit(data.iloc[:,i])
    data1.iloc[:,i] = le.transform(data1.iloc[:,i])
    data2.iloc[:,i] = le.transform(data2.iloc[:,i])
    data3.iloc[:,i] = le.transform(data3.iloc[:,i])
# 輸入 i 是 一個陣列 , 表示那些非數值型態的欄位

In [14]:
# 因為上面已經將資料切成 3 份 , 所以做轉換時, data1, data2, data3 都要轉

In [15]:
data1.dtypes

Unnamed: 0            int64
Title                object
Year                  int64
Brand                object
Location             object
State                object
Rental                 bool
Price               float64
Mileage             float64
Transmission         object
FuelType             object
Engine               object
Drivetrain           object
OptionCount           int64
MajorOptions         object
Accident Check      float64
OwnershipHistory    float64
dtype: object

In [16]:
data1.iloc[:,[1,3,4,5,6,9,10,11,12,14]].dtypes

Title           object
Brand           object
Location        object
State           object
Rental            bool
Transmission    object
FuelType        object
Engine          object
Drivetrain      object
MajorOptions    object
dtype: object

In [17]:
notNumbers = [1,3,4,5,6,9,10,11,12,14]
# 有哪些欄位不是數值型態

In [18]:
data1.iloc[:,notNumbers].dtypes

Title           object
Brand           object
Location        object
State           object
Rental            bool
Transmission    object
FuelType        object
Engine          object
Drivetrain      object
MajorOptions    object
dtype: object

In [19]:
data2.iloc[:,notNumbers].dtypes

Title           object
Brand           object
Location        object
State           object
Rental            bool
Transmission    object
FuelType        object
Engine          object
Drivetrain      object
MajorOptions    object
dtype: object

In [20]:
data3.iloc[:,notNumbers].dtypes

Title           object
Brand           object
Location        object
State           object
Rental            bool
Transmission    object
FuelType        object
Engine          object
Drivetrain      object
MajorOptions    object
dtype: object

In [21]:
for i in notNumbers:
    transform_to_integer(i)

c:\program files\python37\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [22]:
data1.dtypes

Unnamed: 0            int64
Title                 int32
Year                  int64
Brand                 int32
Location              int32
State                 int32
Rental                int64
Price               float64
Mileage             float64
Transmission          int32
FuelType              int32
Engine                int32
Drivetrain            int32
OptionCount           int64
MajorOptions          int32
Accident Check      float64
OwnershipHistory    float64
dtype: object

In [23]:
data2.dtypes

Unnamed: 0            int64
Title                 int32
Year                  int64
Brand                 int32
Location              int32
State                 int32
Rental                int64
Price               float64
Mileage             float64
Transmission          int32
FuelType              int32
Engine                int32
Drivetrain            int32
OptionCount           int64
MajorOptions          int32
Accident Check      float64
OwnershipHistory    float64
dtype: object

In [24]:
data3.dtypes

Unnamed: 0            int64
Title                 int32
Year                  int64
Brand                 int32
Location              int32
State                 int32
Rental                int64
Price               float64
Mileage             float64
Transmission          int32
FuelType              int32
Engine                int32
Drivetrain            int32
OptionCount           int64
MajorOptions          int32
Accident Check      float64
OwnershipHistory    float64
dtype: object

In [25]:
# 已確認 data1 , data2 ,data3 都已經轉成數值型態

--------------------------------

### 3. 將data1 切割成訓練和測試資料

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
data1.head()

,Unnamed: 0,Title,Year,Brand,Location,State,Rental,Price,Mileage,Transmission,FuelType,Engine,Drivetrain,OptionCount,MajorOptions,Accident Check,OwnershipHistory
0,2,10461,2010,8,1976,15,0,15900.0,144000.0,42,4,28,2,2,187537,0.0,2.0
1,3,3183,2014,8,3100,24,0,49995.0,2500.0,32,4,28,4,5,9464,0.0,1.0
2,4,7065,2002,8,1382,12,0,3440.0,174000.0,42,4,22,3,2,41165,0.0,2.0
3,5,7062,2004,8,1382,12,0,4890.0,115500.0,42,4,22,3,1,187595,0.0,3.0
4,7,10273,2001,8,1382,12,0,6550.0,188583.0,42,4,28,4,3,127567,0.0,1.0


In [28]:
y1 = data1.iloc[:,7]
# y 是 Price 欄位

In [29]:
X1 = data1.drop(columns=['Unnamed: 0','Price'])
# X是除了 Price 還有 Unnamed: 0 以外的欄位

In [30]:
X1.head()

,Title,Year,Brand,Location,State,Rental,Mileage,Transmission,FuelType,Engine,Drivetrain,OptionCount,MajorOptions,Accident Check,OwnershipHistory
0,10461,2010,8,1976,15,0,144000.0,42,4,28,2,2,187537,0.0,2.0
1,3183,2014,8,3100,24,0,2500.0,32,4,28,4,5,9464,0.0,1.0
2,7065,2002,8,1382,12,0,174000.0,42,4,22,3,2,41165,0.0,2.0
3,7062,2004,8,1382,12,0,115500.0,42,4,22,3,1,187595,0.0,3.0
4,10273,2001,8,1382,12,0,188583.0,42,4,28,4,3,127567,0.0,1.0


In [31]:
X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y1, test_size=0.3, random_state=1)

In [32]:
y1_train

1012878    38517.0
1406891    15980.0
1380855     9500.0
601250     33990.0
1761264    13171.0
            ...   
519797     57990.0
1936445    18997.0
496761     24898.0
520322      9777.0
132713     32995.0
Name: Price, Length: 1389857, dtype: float64

In [ ]:
# 已經將 data1 切割成訓練和測試資料
# 訓練資料 : X1_train , y1_train
# 測試資料 : X1_test , y1_test

----------------------------------

### 4. 使用隨機森林迴歸 ,估計 Price

In [33]:
from sklearn.ensemble import RandomForestRegressor

In [34]:
model = RandomForestRegressor(n_estimators=10, criterion='mse', random_state=0)

In [35]:
model.fit(X1_train, y1_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [ ]:
model.predict(X1_test)

In [ ]:
y1_pred = model.predict(X1_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y1_test, y1_pred)

In [ ]:
y1_train_pred = model.predict(X1_train)

In [ ]:
r2_score(y1_train, y1_train_pred)

In [ ]:
# 假如讓模型來預測訓練資料 , 也就是 model.predict(X1_train) , 最後的 r平方 會來到 0.954008578414598
# 而如果是以測試資料來做預測 , 也就是 model.predict(X1_test) , 最後的 r 平方 為 0.8125239744153182
# 有過度擬合的狀況

In [ ]:
# 2020/01/14 , 將 Unnamed: 0 欄位去掉 , 再去訓練模型 ,最終 r2 score 反而下降了
# 訓練資料的 r平方 : 0.948170737806687
# 測試資料的 r平方 : 0.7855675609751235

-------------------------------------------

### 5.對 X 和 y 做標準化後,再訓練模型

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
stdsc = StandardScaler()

In [ ]:
X1_train_std = stdsc.fit_transform(X1_train)

In [ ]:
X1_test_std = stdsc.transform(X1_test)

In [ ]:
# y1_train_std = stdsc.fit_transform(y1_train)
# y1_test_std = stdsc.transform(y1_test)
# 因為 欄位(shape) 不同 , 不能用 X1_train fit出來的 stdsc 來 transform y1

In [ ]:
model_std = RandomForestRegressor(n_estimators=10, criterion='mse', random_state=0)

In [ ]:
model_std.fit(X1_train_std, y1_train)

In [ ]:
model_std.predict(X1_test_std)

In [ ]:
y1_pred_std = model_std.predict(X1_test_std)

In [ ]:
r2_score(y1_test, y1_pred_std)

In [ ]:
y1_train_pred_std = model_std.predict(X1_train_std)

In [ ]:
r2_score(y1_train, y1_train_pred_std)

In [ ]:
# 標準化後 , 測試數據的 r 平方 差不多 , 標準化後的 r 平方為 0.7886964137618209
# 訓練數據的 r 平方 為 0.9479330625265767

------------------------------------------

### 6.將 Title 欄位拿掉 , 再去做訓練

In [ ]:
y1 = data1.iloc[:,7]
X1_no_title = data1.drop(columns=['Unnamed: 0','Title','Price'])

In [ ]:
X1_no_title_train, X1_no_title_test, y1_train, y1_test = train_test_split(
    X1_no_title, y1, test_size=0.3, random_state=1)

In [ ]:
model2 = RandomForestRegressor(n_estimators=10, criterion='mse', random_state=0)

In [ ]:
model2.fit(X1_no_title_train, y1_train)

In [ ]:
y1_no_title_pred = model2.predict(X1_no_title_test)

In [ ]:
r2_score(y1_test,y1_no_title_pred)

In [ ]:
# 將 Title 拿掉後 , r平方和不拿掉 Title 差不多,仍然是 0.78 (0.7825372518881237)

In [ ]:
y1_no_title_train_pred = model2.predict(X1_no_title_train) #用訓練資料 predict

In [ ]:
r2_score(y1_train, y1_no_title_train_pred)